In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("../processed_data/final_pre-processed.csv")

In [3]:
df = df.drop('Unnamed: 0',axis=1)[:4000]
df["Ratings"] = df["Ratings"].astype(int)


text_vectorizer = TfidfVectorizer(max_df=.8)
text_vectorizer.fit(df['reviewText'])
def rate(r):
    ary2 = []
    for rating in r:
        tv = [0,0,0,0,0]
        tv[rating-1] = 1
        ary2.append(tv)
    return np.array(ary2)

In [4]:
X = text_vectorizer.transform(df['reviewText']).toarray()
y = rate(df['Ratings'].values)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2)

In [5]:
global_model = Sequential()
global_model.add(Dense(128,input_dim=X_train.shape[1]))
global_model.add(Dense(5,activation='softmax'))

Metal device set to: Apple M1


In [6]:
global_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [7]:
NUM_CLIENTS = 10
EPOCHS_PER_ROUND = 10
BATCH_SIZE = 32

In [8]:
x_train_clients = np.array_split(X_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(X_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

In [12]:
client_models = []
for i in range(NUM_CLIENTS):
    local_model = Sequential()
    local_model.add(Dense(128,input_dim=X_train.shape[1]))
    local_model.add(Dense(5,activation='softmax'))
    local_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    print(acc)
    if(acc[1]>0.69):
        client_models.append(local_model)

3/3 [==============================] - 0s 19ms/step - loss: 0.8142 - accuracy: 0.7000
[0.8142207860946655, 0.699999988079071]
3/3 [==============================] - 0s 19ms/step - loss: 0.9395 - accuracy: 0.6875
[0.9394590258598328, 0.6875]
3/3 [==============================] - 0s 20ms/step - loss: 1.0008 - accuracy: 0.5875
[1.0007925033569336, 0.5874999761581421]
3/3 [==============================] - 0s 22ms/step - loss: 1.0616 - accuracy: 0.6250
[1.0615500211715698, 0.625]
3/3 [==============================] - 0s 22ms/step - loss: 0.8978 - accuracy: 0.7125
[0.8978105783462524, 0.7124999761581421]
3/3 [==============================] - 0s 17ms/step - loss: 0.9650 - accuracy: 0.6625
[0.9650093913078308, 0.6625000238418579]
3/3 [==============================] - 0s 17ms/step - loss: 0.6192 - accuracy: 0.8125
[0.6192106008529663, 0.8125]
3/3 [==============================] - 0s 15ms/step - loss: 0.8652 - accuracy: 0.6875
[0.8652000427246094, 0.6875]
3/3 [=============================

In [13]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(len(client_models)):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [14]:
acc = global_model.evaluate(X_test,y_test)
print(acc)

25/25 [==============================] - 0s 8ms/step - loss: 1.4605 - accuracy: 0.6862
[1.4604908227920532, 0.6862499713897705]


In [15]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(X_train,y_train,epochs=10,batch_size=32)

Epoch 1/10
100/100 [==============================] - 1s 10ms/step - loss: 0.9770 - accuracy: 0.6959
Epoch 2/10
100/100 [==============================] - 1s 9ms/step - loss: 0.7725 - accuracy: 0.7022
Epoch 3/10
100/100 [==============================] - 1s 9ms/step - loss: 0.6664 - accuracy: 0.7416
Epoch 4/10
100/100 [==============================] - 1s 9ms/step - loss: 0.5784 - accuracy: 0.7812
Epoch 5/10
100/100 [==============================] - 1s 10ms/step - loss: 0.4958 - accuracy: 0.8297
Epoch 6/10
100/100 [==============================] - 1s 9ms/step - loss: 0.4240 - accuracy: 0.8700
Epoch 7/10
100/100 [==============================] - 1s 9ms/step - loss: 0.3613 - accuracy: 0.8941
Epoch 8/10
100/100 [==============================] - 1s 9ms/step - loss: 0.3091 - accuracy: 0.9106
Epoch 9/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2660 - accuracy: 0.9228
Epoch 10/10
100/100 [==============================] - 1s 9ms/step - loss: 0.2324 - accuracy: 0.93

In [16]:
acc = fed_model.evaluate(X_test,y_test)
print(acc)

25/25 [==============================] - 0s 7ms/step - loss: 0.9323 - accuracy: 0.6762
[0.9322587847709656, 0.6762499809265137]
